In [49]:
import sympy as sp
from gEconpy.classes.time_aware_symbol import TimeAwareSymbol
import numpy as np

In [62]:
endo_names = ['Y', 'C', 'K', 'N', 'A', 'r' , 'w', 'I']
exo_names = ['epsilon']
param_names = ['beta', 'delta', 'gamma', 'psi', 'alpha', 'rho', 'A_bar']

endo_names_nbr = len(endo_names)
exo_names_nbr = len(exo_names)
param_names_nbr = len(param_names)

endo_vars = [Y, C, K, N, A, r, w, I] = list(map(lambda name: TimeAwareSymbol(name, 0), endo_names))
exo_vars = [epsilon] = list(map(lambda name: TimeAwareSymbol(name, 0), exo_names))

def process_name(s):
    if s.endswith('_bar'):
        return r'\bar{' + s.replace('_bar', '') + '}'
    return s

beta, delta, gamma, psi, alpha, rho, A_bar = list(map(lambda name: sp.Symbol(process_name(name)), param_names))

In [64]:
dynamic_eqs = [
    gamma / C - gamma * beta * C.set_t(1) * (1 - delta - r.set_t(1)),
    w - psi * C / gamma / (1 - N),
    K - (1 - delta) * K.set_t(-1) - I,
    Y - C - I,
    Y - A * K.set_t(-1) ** alpha * N ** (1 - alpha),
    w - (1 - alpha) * Y / N,
    r - alpha * Y / N,
    sp.log(A) - (1 - rho) * A_bar + rho * sp.log(A.set_t(-1)) - epsilon
]

n_eqs = len(dynamic_eqs)

In [111]:
def latex_print_system(system):
    from IPython.display import display, Math

    latex = '\\begin{align}'
    latex += '\\\\'.join([sp.printing.latex(eq) for eq in system])
    latex += '\\end{align}'
    display(Math(latex))

In [112]:
latex_print_system(dynamic_eqs)

<IPython.core.display.Math object>

In [97]:
from functools import reduce
all_symbols = reduce(lambda l,r: l.union(r),
                     [set(eq.atoms()) for eq in dynamic_eqs])
all_symbols = [x for x in all_symbols if isinstance(x, TimeAwareSymbol)]
lag_lead_incidence = np.zeros((3, endo_names_nbr))

for i, var in enumerate(endo_vars):
    time_vec = np.array([var.set_t(t) in all_symbols for t in [-1, 0, 1]])
    lag_lead_incidence[:, i] = time_vec

has_past, has_present, has_future = lag_lead_incidence.astype(bool)
lag_lead_incidence[np.where(lag_lead_incidence)] = np.arange(lag_lead_incidence.sum())

In [102]:
static_vars = np.array(endo_vars)[has_present & (~has_past) & (~has_future)]
predet_vars = np.array(endo_vars)[has_past & (~has_future)]
forward_vars = np.array(endo_vars)[(~has_past) & has_future]
mixed_vars = np.array(endo_vars)[has_past & has_future]

groups = [static_vars, predet_vars, forward_vars, mixed_vars]
names = ['static', 'predetermined', 'forward', 'mixed']

for group, name in zip(groups, names):
    print(f'{name.title()} variables: {", ".join([x.base_name for x in group])}')

Static variables: Y, N, w, I
Predetermined variables: K, A
Forward variables: C, r
Mixed variables: 


In [108]:
endo_vars

[Y_t, C_t, K_t, N_t, A_t, r_t, w_t, I_t]

In [116]:
sp.Matrix(dynamic_eqs).jacobian(endo_vars)

Matrix([
[               0,          -gamma/C_t**2, 0,                                                  0,                              0, 0, 0,  0],
[               0, -psi/(gamma*(1 - N_t)), 0,                      -psi*C_t/(gamma*(1 - N_t)**2),                              0, 0, 1,  0],
[               0,                      0, 1,                                                  0,                              0, 0, 0, -1],
[               1,                     -1, 0,                                                  0,                              0, 0, 0, -1],
[               1,                      0, 0, -A_t*K_t-1**alpha*N_t**(1 - alpha)*(1 - alpha)/N_t, -K_t-1**alpha*N_t**(1 - alpha), 0, 0,  0],
[-(1 - alpha)/N_t,                      0, 0,                             Y_t*(1 - alpha)/N_t**2,                              0, 0, 1,  0],
[      -alpha/N_t,                      0, 0,                                   alpha*Y_t/N_t**2,                              0, 1, 0,  0],
[   

In [114]:
from gEconpy.shared.utilities import eq_to_ss
static_system = [eq_to_ss(eq) for eq in dynamic_eqs]
latex_print_system(static_system)

<IPython.core.display.Math object>

In [115]:
sp.Matrix(static_system).jacobian([x.to_ss() for x in endo_vars])

Matrix([
[                0, -beta*gamma*(-delta - r_ss + 1) - gamma/C_ss**2,                                              0,                                                    0,                              0, beta*gamma*C_ss, 0,  0],
[                0,                         -psi/(gamma*(1 - N_ss)),                                              0,                      -psi*C_ss/(gamma*(1 - N_ss)**2),                              0,               0, 1,  0],
[                0,                                               0,                                          delta,                                                    0,                              0,               0, 0, -1],
[                1,                                              -1,                                              0,                                                    0,                              0,               0, 0, -1],
[                1,                                               0, -alpha*A_s